In [5]:
! pip install requests

You are using pip version 19.0, however version 19.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
! pip install beautifulsoup4

You are using pip version 19.0, however version 19.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [7]:
from bs4 import BeautifulSoup
import requests

page_link = 'http://www.espn.com/nfl/story/_/id/25824920/angry-new-orleans-saints-fans-file-lawsuits-put-billboards'

page_response = requests.get(page_link, timeout = 5)

page_content = BeautifulSoup(page_response.content, "html.parser")

textContent = []
for i in range(0, 26):
    paragraphs = page_content.find_all("p")[i].text
    textContent.append(paragraphs)
    
for i in textContent:
    print (i)

Adam Schefter breaks down Saints owner Gayle Benson's comments about the controversial missed pass interference call in New Orleans' loss to the Rams. (0:58)
METAIRIE, La. -- New Orleans Saints fans are having their say.
After a stunning missed call by officials helped to keep the Saints out of the Super Bowl, angry fans have been reacting in various ways -- from lawsuits and petitions, to billboards in Atlanta, to boycotts throughout the New Orleans area, to a letter from Louisiana's governor that chastised NFL commissioner Roger Goodell.
In the wake of the no-call that helped decide Sunday's NFC Championship Game, the NFL will discuss whether to make pass-interference penalties reviewable, a source confirmed to ESPN.
Sunday's no-call in the NFC championship game shows us why it's time for the NFL to make pass interference a reviewable play.
Sean Payton said the NFL's head of officials told him the officials "blew the call" by not penalizing the Los Angeles Rams for pass interference 

In [12]:
from urllib.request import urlopen
import re
 
html_page = urlopen("http://www.espn.com/nfl/team/_/name/buf/buffalo-bills")
soup = BeautifulSoup(html_page)
for link in soup.find_all('a'):
    print(link.get('href'))

C:\Users\atenk\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\atenk\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


#
/
#
#
/nfl/
/nba/
/mlb/
/soccer/
/nhl/
/mma/
/tennis/
/mens-college-basketball/
/womens-college-basketball/
/college-football/
/golf/
#
/wnba/
/nba-g-league/
/boxing/
/wwe/
/esports/
/chalk/
/analytics/
http://www.espn.com/espnw/sport/ncaa-softball/
/racing/nascar/
/jayski/
/f1/
/racing/
/olympics/
/horse-racing/
http://www.espn.com/college-sports/football/recruiting/
http://www.espn.com/college-sports/basketball/recruiting/
/college-sports/
/little-league-world-series/
http://www.espn.com/specialolympics/
http://xgames.com/
http://espncricinfo.com/
/rugby/
http://www.tsn.ca/cfl
#
/fantasy/
http://www.espn.com/espnradio/index
http://www.espn.com/watch/
http://www.espn.com/watch/espnplus/?om-navmethod=topnav
None
/nfl/team/_/name/buf
/nfl/team/stats/_/name/buf
/nfl/team/schedule/_/name/buf
/nfl/team/roster/_/name/buf
http://www.espn.com/nfl/team/splits/_/name/buf
/nfl/team/depth/_/name/buf
/nfl/team/injuries/_/name/buf
/nfl/team/transactions/_/name/buf
#
/blog/buffalo-bills
https://ww